# Compile a day of miniseed data

In [14]:
from obspy import read
import xarray as xr
import os
from scipy import signal
import numpy as np
from datetime import datetime
import dask.array as da
import multiprocessing

In [2]:
instrument_day_path = "../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05"

## Find all the miniseed files for the selected instrument and day

First define a function that recursively searches for all files with a \*.mseed extension:

In [3]:
def find_mseed_files(directory):
    mseed_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".mseed"):
                mseed_files.append(os.path.join(root, file))
    return mseed_files

In [21]:
def make_xarray(file):
    
    time_string = file.split("YDH-")[1].split('.mseed')[0]
    time_val = datetime.strptime(time_string, '%Y-%m-%dT%H:%M:%S.%fZ')
    
    stream = read(file)
    data = stream.detrend().split().decimate(16).merge(method=1, fill_value="interpolate").decimate(10)
    
    fs = data[0].stats["sampling_rate"]

    # Set spectrogram parameters
    segment_length_seconds = .5
    nperseg = int(segment_length_seconds * fs)
    noverlap_percent = 15
    noverlap = int(nperseg * noverlap_percent/100)

    f, t, Sxx = signal.spectrogram(data[0].data, fs, nperseg = nperseg, noverlap = noverlap)
    array_out = xr.DataArray(np.log10(Sxx), 
                           dims=('frequency', 'seconds'),
                           coords={'frequency':f, 'seconds':t})
    array_out.coords['time'] = time_val

    return array_out

In [37]:
def parallel_process(data, func, sub_segment_length, n_processes=None):
    if n_processes is None:
        n_processes = multiprocessing.cpu_count() 

    pool = multiprocessing.Pool(processes=n_processes)
    
    # Initialize a list to hold the xarray sub-arrays
    results_all = []
    
    # Break into a number of 5-minute segments at a time (30-mins total)
    segments = [data[i:i + sub_segment_length] for i in range(0, len(data), sub_segment_length)]
    for idx, segment in enumerate(segments):
        print( "Segment " + str(idx+1) + " of " + str(len(segments)) )
        results = pool.map(func, segment)
        # Use dask for efficiency
        dask_datasets = [da.chunk({'seconds': 50, 'frequency': 50}) for da in results]
        results_all.append(xr.concat(dask_datasets, dim='time'))
        del results
        
    pool.close() 
    pool.join()

    return results_all

In [40]:
mseed_files = find_mseed_files(instrument_day_path)

In [48]:
mseed_files_sub = mseed_files[0:100]
results = parallel_process(mseed_files_sub, make_xarray, 6)
results_full = xr.concat(results, dim='time').sortby('time')


Segment 1 of 17
Segment 2 of 17
Segment 3 of 17
Segment 4 of 17
Segment 5 of 17
Segment 6 of 17
Segment 7 of 17
Segment 8 of 17
Segment 9 of 17
Segment 10 of 17
Segment 11 of 17
Segment 12 of 17
Segment 13 of 17
Segment 14 of 17
Segment 15 of 17
Segment 16 of 17
Segment 17 of 17


In [50]:
results_full.to_netcdf("test.nc")

In [35]:
test = xr.concat(results, dim='time')
test

<xarray.DataArray (time: 50, frequency: 101, seconds: 705)> Size: 28MB
dask.array<concatenate, shape=(50, 101, 705), dtype=float64, chunksize=(1, 50, 50), chunktype=numpy.ndarray>
Coordinates:
  * frequency  (frequency) float64 808B 0.0 2.0 4.0 6.0 ... 196.0 198.0 200.0
  * seconds    (seconds) float64 6kB 0.25 0.675 1.1 1.525 ... 298.6 299.0 299.4
  * time       (time) datetime64[ns] 400B 2024-01-05T10:30:00 ... 2024-01-05T...

In [2]:
segment_list = []

for idx, file in enumerate(mseed_files):
    # print( "File " + str(idx+1) + " of " + str(len(mseed_files)) )
    stream = read(file)
    dat = stream.detrend().split().decimate(16).merge(method=1, fill_value="interpolate").decimate(10)
    sx = make_xarray(dat)
    segment_list.append(sx)

 

In [ ]:
combined_data = xr.concat(datasets, dim="segment")


In [7]:
f1 = mseed_files[0]
f2 = mseed_files[1]

In [8]:
stream1 = read(f1)
stream2 = read(f2)

In [9]:
dat1 = stream1.detrend().split().decimate(16).merge(method=1, fill_value="interpolate").decimate(10)
dat2 = stream2.detrend().split().decimate(16).merge(method=1, fill_value="interpolate").decimate(10)


In [12]:
sx1 = make_xarray(dat1)

xarray.core.dataarray.DataArray

In [43]:
sub_segment_length = 6
mseed_files_sub = mseed_files[0:12]
segments = [mseed_files_sub[i:i + sub_segment_length] for i in range(0, len(mseed_files_sub), sub_segment_length)]
segments

[['../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T10:30:00.000000Z.mseed',
  '../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T11:50:00.000000Z.mseed',
  '../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T13:00:00.000000Z.mseed',
  '../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T22:30:00.000000Z.mseed',
  '../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T23:35:00.000000Z.mseed',
  '../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T11:25:00.000000Z.mseed'],
 ['../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T03:00:00.000000Z.mseed',
  '../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T04:40:00.000000Z.mseed',
  '../../ooi/san_data/RS03AXBS-LJ03A-09-HYDBBA302/2024/01/05/OO-AXVM1--YDH-2024-01-05T13:05:00.000000Z.mseed',


In [47]:
mseed_files_sub.sort()